In [3]:
# generation ACTION_SPACE based on ACTION_LIST
from rlcard.games.tractor.utils import ACTION_LIST
dict = {ACTION_LIST[i] : i for i in range(len(ACTION_LIST))}
print(dict)

{'3H': 0, '4H': 1, '5H': 2, '6H': 3, '7H': 4, '8H': 5, '9H': 6, 'TH': 7, 'JH': 8, 'QH': 9, 'KH': 10, 'AH': 11, '3C': 12, '4C': 13, '5C': 14, '6C': 15, '7C': 16, '8C': 17, '9C': 18, 'TC': 19, 'JC': 20, 'QC': 21, 'KC': 22, 'AC': 23, '3D': 24, '4D': 25, '5D': 26, '6D': 27, '7D': 28, '8D': 29, '9D': 30, 'TD': 31, 'JD': 32, 'QD': 33, 'KD': 34, 'AD': 35, '3J': 36, '4J': 37, '5J': 38, '6J': 39, '7J': 40, '8J': 41, '9J': 42, 'TJ': 43, 'JJ': 44, 'QJ': 45, 'KJ': 46, 'AJ': 47, 'NH': 48, 'NC': 49, 'ND': 50, 'NJ': 51, 'BJ': 52, 'RJ': 53, '3H,3H': 54, '4H,4H': 55, '5H,5H': 56, '6H,6H': 57, '7H,7H': 58, '8H,8H': 59, '9H,9H': 60, 'TH,TH': 61, 'JH,JH': 62, 'QH,QH': 63, 'KH,KH': 64, 'AH,AH': 65, '3C,3C': 66, '4C,4C': 67, '5C,5C': 68, '6C,6C': 69, '7C,7C': 70, '8C,8C': 71, '9C,9C': 72, 'TC,TC': 73, 'JC,JC': 74, 'QC,QC': 75, 'KC,KC': 76, 'AC,AC': 77, '3D,3D': 78, '4D,4D': 79, '5D,5D': 80, '6D,6D': 81, '7D,7D': 82, '8D,8D': 83, '9D,9D': 84, 'TD,TD': 85, 'JD,JD': 86, 'QD,QD': 87, 'KD,KD': 88, 'AD,AD': 89, '

In [1]:
# Generate C# action space dictionary

from rlcard.games.tractor.utils import ACTION_SPACE

s = ["\"{}\", {}".format(x, ACTION_SPACE[x]) for x in ACTION_SPACE]

s = [("{" + x + "}") for x in s]

print(', '.join(s))

{"3H", 0}, {"4H", 1}, {"5H", 2}, {"6H", 3}, {"7H", 4}, {"8H", 5}, {"9H", 6}, {"TH", 7}, {"JH", 8}, {"QH", 9}, {"KH", 10}, {"AH", 11}, {"3C", 12}, {"4C", 13}, {"5C", 14}, {"6C", 15}, {"7C", 16}, {"8C", 17}, {"9C", 18}, {"TC", 19}, {"JC", 20}, {"QC", 21}, {"KC", 22}, {"AC", 23}, {"3D", 24}, {"4D", 25}, {"5D", 26}, {"6D", 27}, {"7D", 28}, {"8D", 29}, {"9D", 30}, {"TD", 31}, {"JD", 32}, {"QD", 33}, {"KD", 34}, {"AD", 35}, {"3J", 36}, {"4J", 37}, {"5J", 38}, {"6J", 39}, {"7J", 40}, {"8J", 41}, {"9J", 42}, {"TJ", 43}, {"JJ", 44}, {"QJ", 45}, {"KJ", 46}, {"AJ", 47}, {"NH", 48}, {"NC", 49}, {"ND", 50}, {"NJ", 51}, {"BJ", 52}, {"RJ", 53}, {"3H,3H", 54}, {"4H,4H", 55}, {"5H,5H", 56}, {"6H,6H", 57}, {"7H,7H", 58}, {"8H,8H", 59}, {"9H,9H", 60}, {"TH,TH", 61}, {"JH,JH", 62}, {"QH,QH", 63}, {"KH,KH", 64}, {"AH,AH", 65}, {"3C,3C", 66}, {"4C,4C", 67}, {"5C,5C", 68}, {"6C,6C", 69}, {"7C,7C", 70}, {"8C,8C", 71}, {"9C,9C", 72}, {"TC,TC", 73}, {"JC,JC", 74}, {"QC,QC", 75}, {"KC,KC", 76}, {"AC,AC", 77}, {"

In [ ]:
from rlcard.games.tractor import Dealer

dealer = Dealer('3J', None)

print(dealer.deck)
print(len(dealer.deck))

In [ ]:

import numpy as np

np_random = np.random.RandomState()

from rlcard.games.tractor import Dealer, Round, Player

players = [Player(0, np_random), Player(1, np_random), Player(2, np_random), Player(3, np_random)]

round = Round(np_random)
round.initiate(players, predefined_trump='5H')

round.current_round = [['6J'],['2S'],['NC'],['NJ']]
print(round.calc_score_in_round())


In [ ]:
from rlcard.games.tractor.utils import CARD_RANK_STR

dict = {CARD_RANK_STR[i] : i for i in range(len(CARD_RANK_STR))}

print(dict)

In [ ]:
current_hand = ['2S', '3S', '7S', 'KS', 'AS', '3H', '4H', '5H', '7H', '7H', 'AH', '2D', '4D', 'TD', 'AD', 'AD', '3J', '7J', '8J', 'JJ', 'AJ', 'AJ', 'NS', 'ND', 'NJ']
print(Judger.playable_cards_from_hand(current_hand, '9C'))

In [ ]:
from rlcard.games.tractor import Dealer, Judger
from rlcard.games.tractor.utils import TRUMP_CANDIDATE_STR

action_list = []

for trump in TRUMP_CANDIDATE_STR:
    dealer = Dealer(trump, None)
    current_hand = dealer.deck
    playable_cards = Judger.playable_cards_from_hand(current_hand, trump)
    for cards in playable_cards:
        if cards not in action_list:
            action_list.append(cards)

action_list_str = [','.join(action) for action in action_list]
action_list_str.append('pass')
action_list_str.append('pass_score')
print(action_list_str)
print()

action_dict = {action_list_str[i] : i for i in range(len(action_list_str))}
print(action_dict)


In [9]:
''' Test get_pass_cards_sequence
'''

from rlcard.games.tractor.utils import get_pass_cards_sequence
from rlcard.games.tractor import Judger

trump = 'KC'
score = True
pair_first = True
current_hand = ['2S', '3S', '3S', '5S', 'TS', 'KS', '2H', '3H', '3H', '5H', 'TH', 'KH', '2J', '3J', '3J', '5J', 'TJ', 'TJ', 'KJ', 'NJ', '4D']
playable_cards = Judger.playable_cards_from_hand(current_hand, trump)
units = get_pass_cards_sequence(current_hand, playable_cards, score, pair_first, [0,4,2,3,1], trump)
print(units)
ret = [item for sublist in units for item in sublist]
print(ret)

['3S', '3S', 'TS', 'KS', '5S', '2S', 'KJ', 'NJ', '5J', 'TJ', 'TJ', '2J', '3J', '3J', '4D', 'TH', 'KH', '5H', '2H', '3H', '3H']
['3', 'S', '3', 'S', 'T', 'S', 'K', 'S', '5', 'S', '2', 'S', 'K', 'J', 'N', 'J', '5', 'J', 'T', 'J', 'T', 'J', '2', 'J', '3', 'J', '3', 'J', '4', 'D', 'T', 'H', 'K', 'H', '5', 'H', '2', 'H', '3', 'H', '3', 'H']
